In [1]:
import io
import numpy as np
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from datetime import datetime

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [3]:
buffer_size = 70000
batch_size = 128
num_epochs = 20

In [4]:
mnist_train,mnist_test = mnist_dataset['train'],mnist_dataset['test']

In [5]:
def scaler(image,label):
    image = tf.cast(image,tf.float32)
    image /= 255.
    return image,label

In [6]:
scaled_train_val = mnist_train.map(scaler)
scaled_test = mnist_test.map(scaler)

In [7]:
num_val_samples = 0.1 * mnist_info.splits['train'].num_examples
num_val_samples = tf.cast(num_val_samples,tf.int64)
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

In [8]:
shuffled_train = scaled_train_val.shuffle(buffer_size)
validation_data = scaled_train_val.take(num_val_samples)
train_data = scaled_train_val.skip(num_val_samples)
train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_val_samples)

In [9]:
test_data = scaled_test.batch(num_test_samples)

In [10]:
# Extracting the numpy arrays from the validation data for the calculation of the Confusion Matrix
for images, labels in validation_data:
    images_val = images.numpy()
    labels_val = labels.numpy()

In [11]:
con_model = tf.keras.Sequential([
                                 tf.keras.layers.Conv2D(50,5,activation='relu',input_shape=(28,28,1)),
                                 tf.keras.layers.MaxPooling2D(),
                                 tf.keras.layers.Conv2D(50,3,activation='relu'),
                                 tf.keras.layers.MaxPooling2D(),
                                 tf.keras.layers.Flatten(),
                                 tf.keras.layers.Dense(10)
                                ])

In [12]:
con_model.summary(line_length=75)

Model: "sequential"
___________________________________________________________________________
 Layer (type)                    Output Shape                  Param #     
 conv2d (Conv2D)                 (None, 24, 24, 50)            1300        
                                                                           
 max_pooling2d (MaxPooling2D)    (None, 12, 12, 50)            0           
                                                                           
 conv2d_1 (Conv2D)               (None, 10, 10, 50)            22550       
                                                                           
 max_pooling2d_1 (MaxPooling2D)  (None, 5, 5, 50)              0           
                                                                           
 flatten (Flatten)               (None, 1250)                  0           
                                                                           
 dense (Dense)                   (None, 10)                    12510

In [13]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [14]:
con_model.compile(optimizer='adam',loss=loss,metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='auto',min_delta=0,patience = 2,verbose=0,restore_best_weights=True)

In [15]:
log_dir = "Logs\\fit\\" + "run-1"

In [16]:
def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(12, 12))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    return figure

In [17]:
def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    
    # Closing the figure prevents it from being displayed directly inside the notebook.
    plt.close(figure)
    
    buf.seek(0)
    
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    
    return image

In [21]:
# Define a file writer variable for logging purposes
file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')

def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = con_model.predict(images_val)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = confusion_matrix(labels_val, test_pred)
    
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [22]:
# Defining the callbacks
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)

In [23]:
con_model.fit(train_data,
          epochs = num_epochs,
          validation_data = validation_data,
          verbose = 2,
          callbacks = [early_stopping,tensorboard_callback,cm_callback]
         )

Epoch 1/20
422/422 - 40s - loss: 0.0725 - accuracy: 0.9782 - val_loss: 0.0702 - val_accuracy: 0.9793 - 40s/epoch - 94ms/step
Epoch 2/20
422/422 - 38s - loss: 0.0527 - accuracy: 0.9838 - val_loss: 0.0614 - val_accuracy: 0.9830 - 38s/epoch - 91ms/step
Epoch 3/20
422/422 - 47s - loss: 0.0421 - accuracy: 0.9875 - val_loss: 0.0567 - val_accuracy: 0.9847 - 47s/epoch - 112ms/step
Epoch 4/20
422/422 - 46s - loss: 0.0352 - accuracy: 0.9895 - val_loss: 0.0540 - val_accuracy: 0.9855 - 46s/epoch - 109ms/step
Epoch 5/20
422/422 - 45s - loss: 0.0299 - accuracy: 0.9912 - val_loss: 0.0514 - val_accuracy: 0.9863 - 45s/epoch - 107ms/step
Epoch 6/20
422/422 - 46s - loss: 0.0254 - accuracy: 0.9926 - val_loss: 0.0498 - val_accuracy: 0.9862 - 46s/epoch - 110ms/step
Epoch 7/20
422/422 - 45s - loss: 0.0218 - accuracy: 0.9939 - val_loss: 0.0480 - val_accuracy: 0.9867 - 45s/epoch - 108ms/step
Epoch 8/20
422/422 - 46s - loss: 0.0186 - accuracy: 0.9951 - val_loss: 0.0486 - val_accuracy: 0.9873 - 46s/epoch - 109ms

In [24]:
test_loss,test_accuracy = con_model.evaluate(test_data)
print(f"Loss = {round(test_loss,2)} , accuracy = {round(test_accuracy*100,2)} %")

1/1 [==============================] - 2s 2s/step - loss: 0.0332 - accuracy: 0.9888
Loss = 0.03 , accuracy = 98.88 %


## Tensorboard

In [25]:
%load_ext tensorboard
%tensorboard --logdir "logs/fit"

Reusing TensorBoard on port 6006 (pid 11400), started 0:51:49 ago. (Use '!kill 11400' to kill it.)